In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm 
#%matplotlib notebook
from matplotlib import pyplot as plt
print(cv2.__version__)

In [ ]:
INPUT_PATH="../imageDatabase/outputImages/96/"
OUTPUT_PATH="./lsbImages/"

In [ ]:
def showRGBImg(img:np.ndarray):
    imgShow = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(num='fg1',figsize=(6,6))
    plt.imshow(imgShow)
    plt.show()

In [ ]:
def convertStringToBinaryCode(message:str,img_height:int,img_width:int):
    ans=[]
    for word in message:
        str=format(ord(word),'08b')
        for binary in str:
            ans.append(int(binary))

    if len(ans)>img_width*img_height*3:
        print('Message is too long')
        return None
    
    while len(ans)<img_width*img_height*3:
        ans.append(0)

    return ans

In [ ]:
def LSB_encode(img:np.ndarray, message:str):
    img_height=img.shape[0]
    img_width =img.shape[1]
    
    img_written=img.copy()

    writtenData=convertStringToBinaryCode(message,img_height,img_width)
    
    index=0
    for k in range(0,3):
        for i in range(img_height):
            for j in range(img_width):
                if writtenData[index]==1:
                    img_written[i,j,k]=img_written[i,j,k] | 1
                else:
                    img_written[i,j,k]=img_written[i,j,k] & ~1
                index+=1

    return img_written

In [ ]:
def LSB_decode(img:np.ndarray):

    img_height=img.shape[0]
    img_width =img.shape[1]
    
    writtenData=[]
    for k in range(0,3):
        for i in range(img_height):
            for j in range(img_width):
                writtenData.append(img[i,j,k] & 1)

    message=''
    letter=''
    for i in writtenData:
        letter+=str(i)
        if len(letter)==8:
            flag=0
            for j in letter:
                if j=='1':
                    flag=1
                    break
            if flag==0:
                continue

            message+=chr(int(letter,2))
            letter=''
            
    return message

In [ ]:
file_list=os.listdir(INPUT_PATH)
for file in tqdm(file_list):
    if file.endswith('.png') or file.endswith('.jpg'):
        img = cv2.imread(INPUT_PATH+file, cv2.IMREAD_COLOR)
        written_msg="[Hidden message: "+file+"]"
        encode_img=LSB_encode(img,written_msg)
        cv2.imwrite(OUTPUT_PATH+"_"+file[:-4]+'_written.png', encode_img, [cv2.IMWRITE_PNG_COMPRESSION, 1])

In [ ]:
file_list=os.listdir(OUTPUT_PATH)
for file in tqdm(file_list):
    if file.endswith('.png') or file.endswith('.jpg'):
        img = cv2.imread(OUTPUT_PATH+file, cv2.IMREAD_COLOR)
        decode_msg=LSB_decode(img)
        # print("decode message:", decode_msg)